In [1]:
from feat import Detector
from PIL import Image, ImageDraw, ImageFont
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf

In [2]:
# Initialize Py-Feat detector
face_model = "retinaface"
landmark_model = "mobilenet"
au_model = "xgb"
emotion_model = "resmasknet"
detector = Detector(face_model=face_model, landmark_model=landmark_model, au_model=au_model, emotion_model=emotion_model)

# Directories for input images and output CSV file
image_dir = 'DiffusionFER/DiffusionEmotion_S/cropped'   
aus_csv = 'aus_activations.csv'

INFO:root:Loading Face model: retinaface
/Users/abhinavramalingam/Documents/Uppsala/IID/.conda/lib/python3.10/site-packages/feat/face_detectors/Retinaface/Retinaface_test.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for 

In [3]:
def process_image(image_path, emotion_label):
    image = Image.open(image_path)
    image_prediction = detector.detect_image(image_path, data_type="image")
    
    image_aus_data = []
    
    if not image_prediction.empty:  # Check if a face is detected
        row = image_prediction.iloc[0]  # Take the first detected face
        face_aus = image_prediction.aus.iloc[0]  # AUs for this face
        
        # Collect AU activations
        for au_col, activation in face_aus.items():
            image_aus_data.append([image_path, au_col, activation, emotion_label])
    
    return image_aus_data

In [4]:
def accumulate_aus_data():
    aus_data = []
    
    # Iterate over each emotion subdirectory (anger, disgust, etc.)
    for emotion_label in os.listdir(image_dir):
        emotion_dir = os.path.join(image_dir, emotion_label)
        if os.path.isdir(emotion_dir):
            for image_filename in os.listdir(emotion_dir):
                if image_filename.endswith(".jpg") or image_filename.endswith(".png"):   
                    # Get the full path of the image
                    image_path = os.path.join(emotion_dir, image_filename)

                    # Process the image and accumulate AU data
                    image_aus_data = process_image(image_path, emotion_label)

                    # Append the AU data to the global list
                    for data in image_aus_data:
                        aus_data.append(data)

                    # Save AU activations to CSV immediately after processing
                    if aus_data:
                        aus_df = pd.DataFrame(aus_data, columns=["file", "AU", "activation", "emotion"])
                        aus_df.to_csv(aus_csv, mode='a', header=not os.path.exists(aus_csv), index=False)

    print(f"AU activations saved to {aus_csv}")

In [5]:
# Run the accumulation process
accumulate_aus_data()

  0%|          | 0/1 [00:00<?, ?it/s]INFO:root:detecting faces...
INFO:root:detecting landmarks...
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 220, 223]) to 224
INFO:root:detecting poses...
INFO:root:img2pose: RESCALING WARNING: img2pose has a min img size of 400 and a max img size of 1400 but checked value is torch.Size([224, 224]).
INFO:root:detecting aus...
INFO:root:detecting emotions...
INFO:root:inverting face transform...
INFO:root:inverting landmark transform...
INFO:root:creating fex output...
  0%|          | 0/1 [00:00<?, ?it/s]INFO:root:detecting faces...
INFO:root:detecting landmarks...
INFO:root:RESCALING WARNING: image_operations.extract_face_from_bbox() is rescaling cropped img with shape torch.Size([3, 224, 224]) to 224
INFO:root:detecting poses...
INFO:root:img2pose: RESCALING WARNING: img2pose has a min img size of 400 and a max img size of 1400 but checked value is torch.Size([224, 224]).


AU activations saved to aus_activations.csv
